In [1]:
import pandas as pd
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
market = Market()
spec = ADatabase("forex_classification")
umod = UniversalModeler()

In [3]:
market.connect()
data = market.retrieve("forex")
market.disconnect()

In [4]:
factors = []
factors.append("d1")
factors.append("d2")
factors.append("d3")
factors.append("rolling14")

In [5]:
included_columns=[
    "year",
    "week",
    "ticker",
    "adjclose",
    "y"
]

In [6]:
included_columns.extend(factors)

In [7]:
training_sets = []

In [8]:
all_prices = data.melt(id_vars="date",value_vars=[x for x in data.columns if x != "date"]).rename(columns={"variable":"ticker","value":"adjclose"})

In [9]:
all_prices

,date,ticker,adjclose
0,2000-01-01,ANG,1.790000
1,2000-01-02,ANG,1.790000
2,2000-01-03,ANG,1.790000
3,2000-01-04,ANG,1.790000
4,2000-01-05,ANG,1.790000
...,...,...,...
1446015,2023-04-11,VES,24.361623
1446016,2023-04-12,VES,24.442814
1446017,2023-04-13,VES,24.457308
1446018,2023-04-14,VES,24.511235


In [10]:
for ticker in tqdm(all_prices["ticker"].unique()):
    try:
        prices = all_prices[all_prices["ticker"]==ticker]
        prices = p.column_date_processing(prices)
        prices["year"] = [x.year for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        ticker_data = prices[prices["ticker"]==ticker]
        ticker_data.sort_values("date",ascending=True,inplace=True)
        ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
        ticker_data = ticker_data.groupby(["year","week"]).mean().reset_index()
        ticker_data["d1"] = ticker_data["adjclose"].pct_change(periods=1)
        ticker_data["d2"] = ticker_data["d1"].pct_change(periods=1)
        ticker_data["d3"] = ticker_data["d2"].pct_change(periods=1)
        ticker_data["rolling14"] = ticker_data["adjclose"].rolling(window=14).mean()
        ticker_data.dropna(inplace=True)
        ticker_data["ticker"] = ticker
        ticker_data["future"] = ticker_data["adjclose"].shift(-1)
        ticker_data["delta"] = (ticker_data["future"] - ticker_data["adjclose"]) / ticker_data["adjclose"]
        ticker_data["y"] = [x > 0 for x in ticker_data["delta"]]
        ticker_data = ticker_data.replace([np.inf, -np.inf], np.nan).dropna()
        ticker_data.dropna(inplace=True)
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except:
        continue

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 170/170 [00:26<00:00,  6.53it/s]


In [11]:
data = pd.concat(training_sets)

In [12]:
data = data.dropna()

In [13]:
data

,year,week,ticker,adjclose,y,d1,d2,d3,rolling14
52,2001,1,ANG,1.790000,1.0,0.000000e+00,-1.000000,-1.000000,1.790000
54,2001,3,ANG,1.790000,0.0,0.000000e+00,-1.000000,-1.000000,1.790000
105,2002,2,ANG,1.790000,0.0,2.220446e-16,-3.000000,-1.000000,1.790000
106,2002,3,ANG,1.790000,0.0,0.000000e+00,-1.000000,-0.666667,1.790000
157,2003,2,ANG,1.790000,0.0,2.220446e-16,-3.000000,-1.000000,1.790000
...,...,...,...,...,...,...,...,...,...
1214,2023,11,VES,24.061629,1.0,-3.372588e-03,-0.552261,-1.200980,20.955136
1215,2023,12,VES,24.095671,1.0,1.414796e-03,-1.419498,1.570342,21.618622
1216,2023,13,VES,24.426026,1.0,1.371014e-02,8.690544,-7.122264,22.202572
1217,2023,14,VES,24.449670,1.0,9.679839e-04,-0.929396,-1.106943,22.751233


In [ ]:
spec.connect()
years = 4
for modeler in [umod]:
    for year in tqdm(range(2019,2023)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.classification_model(training_slice,prediction_set,factors,multioutput=False)
        stuff["training_years"] = years
        spec.store(set_name,stuff)
spec.disconnect()

  0%|                                                                                                                      | 0/4 [00:00<?, ?it/s]

Epoch 1/10
773/773 [==============================] - 1s 859us/step - loss: 100235696.0000 - accuracy: 0.5003
Epoch 2/10
773/773 [==============================] - 1s 812us/step - loss: 5354658.5000 - accuracy: 0.4977
Epoch 3/10
773/773 [==============================] - 1s 805us/step - loss: 24905042.0000 - accuracy: 0.4919
Epoch 4/10
773/773 [==============================] - 1s 798us/step - loss: 145908384.0000 - accuracy: 0.4778
Epoch 5/10
773/773 [==============================] - 1s 794us/step - loss: 77502936.0000 - accuracy: 0.4745
Epoch 6/10
773/773 [==============================] - 1s 802us/step - loss: 5066.0415 - accuracy: 0.4763
Epoch 7/10
773/773 [==============================] - 1s 792us/step - loss: 39362124.0000 - accuracy: 0.4744
Epoch 8/10
773/773 [==============================] - 1s 799us/step - loss: 4159.9458 - accuracy: 0.4765
Epoch 9/10
773/773 [==============================] - 1s 802us/step - loss: 17472658.0000 - accuracy: 0.4762
Epoch 10/10
257/257 [=====

 25%|███████████████████████████▎                                                                                 | 1/4 [02:50<08:30, 170.01s/it]

Epoch 1/10
770/770 [==============================] - 1s 783us/step - loss: 886502912.0000 - accuracy: 0.5037
Epoch 2/10
770/770 [==============================] - 1s 777us/step - loss: 537783744.0000 - accuracy: 0.5043
Epoch 3/10
770/770 [==============================] - 1s 766us/step - loss: 28180642.0000 - accuracy: 0.4950
Epoch 4/10
770/770 [==============================] - 1s 767us/step - loss: 186918624.0000 - accuracy: 0.4873
Epoch 5/10
770/770 [==============================] - 1s 765us/step - loss: 5197255.0000 - accuracy: 0.4916
Epoch 6/10
770/770 [==============================] - 1s 775us/step - loss: 57628744.0000 - accuracy: 0.4895
Epoch 7/10
770/770 [==============================] - 1s 771us/step - loss: 60930.3906 - accuracy: 0.4903
Epoch 8/10
770/770 [==============================] - 1s 772us/step - loss: 3527481.2500 - accuracy: 0.4884
Epoch 9/10
770/770 [==============================] - 1s 788us/step - loss: 50513.0664 - accuracy: 0.4893
Epoch 10/10
261/261 [===

 50%|██████████████████████████████████████████████████████▌                                                      | 2/4 [05:10<05:05, 152.76s/it]

Epoch 1/10
770/770 [==============================] - 1s 807us/step - loss: 26272.0293 - accuracy: 0.5048
Epoch 2/10
770/770 [==============================] - 1s 787us/step - loss: 2442954.2500 - accuracy: 0.5058
Epoch 3/10
770/770 [==============================] - 1s 799us/step - loss: 50879.7461 - accuracy: 0.4917
Epoch 4/10
770/770 [==============================] - 1s 787us/step - loss: 437.7412 - accuracy: 0.4900
Epoch 5/10
770/770 [==============================] - 1s 778us/step - loss: 2229877.5000 - accuracy: 0.4880
Epoch 6/10
770/770 [==============================] - 1s 774us/step - loss: 3288794.5000 - accuracy: 0.4895
Epoch 7/10
770/770 [==============================] - 1s 773us/step - loss: 54625.6406 - accuracy: 0.4893
Epoch 8/10
770/770 [==============================] - 1s 801us/step - loss: 25745.1523 - accuracy: 0.4887
Epoch 9/10
770/770 [==============================] - 1s 798us/step - loss: 11210.2881 - accuracy: 0.4875
Epoch 10/10
253/253 [=====================

 75%|█████████████████████████████████████████████████████████████████████████████████▊                           | 3/4 [07:31<02:27, 147.08s/it]